# TensorFlow
Full tutorial from tensorflow website.
https://www.tensorflow.org/guide/low_level_intro#training

## Graph
Build the computation graph, the blue print

In [2]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import tensorflow as tf
a = tf.constant(3.0, dtype=tf.float32)
b = tf.constant(4.0) # also tf.float32 implicitly
total = a + b
print(a)
print(b)
print(total)


Tensor("Const_2:0", shape=(), dtype=float32)
Tensor("Const_3:0", shape=(), dtype=float32)
Tensor("add_1:0", shape=(), dtype=float32)


## Tensor board
A utility to visualize a computation graph.

In [40]:
writer = tf.summary.FileWriter('.')
writer.add_graph(tf.get_default_graph())
writer.flush()

The output should be as shown below where last two extensions are { timestamp } and { hostname }
    
```events.out.tfevents.{1563999962}.{PHSX-CMS}```

Then type in the terminal

```tensorboard --logdir=data/ --host localhost --port 6006```

Your terminal will show the output

```TensorBoard 1.10.0 at http://localhost:6006 (Press CTRL+C to quit)```

Then type the http address to your browser, and run the following in your terminal.

```tensorboard --logdir .```
 
Refresh the browser and you will see a graph. For my specific test, my browser node is <http://localhost:6006/#graphs&run=.>

## Session

A session runs ```TensorFlow``` operations.

In [18]:
sess = tf.Session()
print(sess.run(total))

7.0


Now, one can write dictionary, list or tuple

In [5]:
print(sess.run({'ab':(a, b), 'total':total}))

{'ab': (3.0, 4.0), 'total': 7.0}


In [9]:
vec = tf.random_uniform(shape=(3,)) #produce tf.Tensor (random 3-element vector with values in [0,1))
print(sess.run(vec))
print(sess.run(vec),'\n')
out1 = vec + 1
out2 = vec + 2
print(sess.run((out1, out2))) #consistent value during a single run

[0.9356557 0.3115723 0.8405701]
[0.72359276 0.82833993 0.66773427] 

(array([1.8219849, 1.6984061, 1.2959051], dtype=float32), array([2.8219848, 2.6984062, 2.295905 ], dtype=float32))


## Feeding
Parameterize a graph to accept external inputs, known as placeholders (equivalent of declaring a functions)

In [10]:
x = tf.placeholder(tf.float32)
y = tf.placeholder(tf.float32)
z = x + y

In [11]:
print(sess.run(z, feed_dict={x: 3, y: 4.5}))
print(sess.run(z, feed_dict={x: [1, 3], y: [2, 4]}))

7.5
[3. 7.]


## Datasets

Placeholders work for simple experiments, but  ```tf.data``` are the preferred method of streaming data into a model.

In [21]:
my_data = [
    [0, 1,],
    [2, 3,],
    [4, 5,],
    [6, 7,],
]
slices = tf.data.Dataset.from_tensor_slices(my_data)#Dataset must be first converted into a tf.data.Iterator
next_item = slices.make_one_shot_iterator().get_next()#Call the Iterator's tf.data.Iterator.get_next method.
while True:
  try:
    print(sess.run(next_item))
  except tf.errors.OutOfRangeError:
    break

[0 1]
[2 3]
[4 5]
[6 7]


In [22]:
r = tf.random_normal([10,3]) #shape=(10,3)
dataset = tf.data.Dataset.from_tensor_slices(r)
iterator = dataset.make_initializable_iterator()
next_row = iterator.get_next()

sess.run(iterator.initializer) #if Dataset depends on stateful operations
while True:
  try:
    print(sess.run(next_row))
  except tf.errors.OutOfRangeError:
    break

[ 0.20655905 -0.61874175 -0.41444466]
[-0.817075   -0.34945825  1.0081652 ]
[ 0.15904525 -1.0414507  -0.3472652 ]
[-2.1826444  -0.77140415  0.8847609 ]
[-0.06684798  0.5613106  -0.50886106]
[-0.43571225  0.9264167  -1.0658191 ]
[-0.17708409  1.1292535  -1.1213037 ]
[-0.76154757  2.0042195   0.6748948 ]
[-1.2873728 -0.2192565 -0.831202 ]
[-0.5123091  0.9066346 -1.3900918]


## Layers
A method to modify the values in the graph to get new outputs with the same input in a trainable model. It adds trainable parameters to a graph by packing together both the variables and the operations that act on them.

In [41]:
# define a placeholder with a shape so a layer can build a weight matrix of the correct size
x = tf.placeholder(tf.float32, shape=[None, 3])  
# create a densely-connected layer (tf.layers.Dense) that takes a batch of input vectors, and 
# produces a single output value (weighted sum across all inputs, with optional activation function)
linear_model = tf.layers.Dense(units=1)
# call the layer as if it were a function
y = linear_model(x)

# initialize
init = tf.global_variables_initializer() 
sess.run(init)

# execute
print(sess.run(y, {x: [[1, 2, 3],[4, 5, 6]]})) 

[[-0.78715086]
 [-0.5930755 ]]


In [42]:
# example of shortcut 
x = tf.placeholder(tf.float32, shape=[None, 3])
# create and run the layer in a single call
y = tf.layers.dense(x, units=1)

# initialize
init = tf.global_variables_initializer()
sess.run(init)

# execute
print(sess.run(y, {x: [[1, 2, 3], [4, 5, 6]]}))

[[2.095778 ]
 [5.2557964]]


Downfall: this approach allows no access to the ```tf.layers.Layer``` object. This makes introspection and debugging more difficult, and layer reuse impossible.